In [40]:
import csv
import time
import sys
import matplotlib.pyplot as plt
import numpy as np
import math
import os.path
from scipy.stats import multivariate_normal


In [102]:
HEIGHT_THRESHOLD = 0.0  # meters
GROUND_HEIGHT_THRESHOLD = -.4  # meters
DT = 0.1
X_LANDMARK = 5.  # meters
Y_LANDMARK = -5.  # meters
EARTH_RADIUS = 6.3781E6  # meters

In [5]:
data17, _ = load_data("2020_2_26__17_21_59_filtered")

In [6]:
#accleration covariance calculations
#assuming uncorrelated Accelx, accelY
var_accX = np.var(data17["AccelX"][669:])
var_accY = np.var(data17["AccelY"][669:])
covar_u_t = np.array([[var_accX, 0],
                      [0, var_accY]])
print(var_accX, var_accY)

0.2597510387811634 0.14175408587257618


In [7]:
yaw_corrected = np.zeros(len(data17["Yaw"][669:]))
yaw_data = data17["Yaw"][669:]

for i in range(len(data17["Yaw"][669:])):
    
    yaw_corrected[i] = wrap_to_pi(yaw_data[i]*(np.pi/180))

yaw_corrected

array([ 0.01090831,  0.        ,  0.00109083, -0.00436332,  0.        ,
       -0.00109083,  0.00872665,  0.00436332,  0.00654498,  0.00545415,
        0.00654498,  0.00654498,  0.00763582,  0.00654498,  0.00654498,
        0.00654498,  0.00654498,  0.00545415,  0.00545415,  0.00545415,
        0.00545415,  0.00545415,  0.00654498,  0.00654498,  0.00654498,
        0.00654498,  0.00763582,  0.00763582,  0.00763582,  0.00763582,
        0.00763582,  0.00763582,  0.00763582,  0.00763582,  0.00763582,
        0.00763582,  0.00763582,  0.00763582])

In [8]:
#calculating sensor covariance matrix. We assume that all measurements are independent, and that the covariance is equal
#to the stationary variance
#recall: measurement vector is Zx, Zy, yaw
var_Zx = np.var(data17["X"][669:])
var_Zy = np.var(data17["Y"][669:])
var_yaw = np.var(yaw_corrected)
covar_z = np.array([[var_Zx, 0, 0],
                    [ 0, var_Zy, 0],
                    [0, 0, var_yaw]])
print(var_Zx, var_Zy, var_yaw)

0.0001698159186405823 0.0004478908815990326 8.613675903064495e-06


In [143]:
def load_data(filename):
    """Load data from the csv log

    Parameters:
    filename (str)  -- the name of the csv log

    Returns:
    data (dict)     -- the logged data with data categories as keys
                       and values list of floats
    """
    is_filtered = False
    if os.path.isfile(filename + "_filtered.csv"):
        f = open(filename + "_filtered.csv")
        is_filtered = True
    else:
        f = open(filename + ".csv")

    file_reader = csv.reader(f, delimiter=',')

    # Load data into dictionary with headers as keys
    data = {}
    header = ["X", "Y", "Z", "Time Stamp", "Latitude", "Longitude",
              "Yaw", "Pitch", "Roll", "AccelX", "AccelY", "AccelZ"]
    for h in header:
        data[h] = []

    row_num = 0
    f_log = open("bad_data_log.txt", "w")
    for row in file_reader:
        for h, element in zip(header, row):
            # If got a bad value just use the previous value
            try:
                data[h].append(float(element))
            except ValueError:
                data[h].append(data[h][-1])
                f_log.write(str(row_num) + "\n")

        row_num += 1
    f.close()
    f_log.close()

    return data, is_filtered

In [144]:
def wrap_to_pi(angle):
    """Wrap angle data in radians to [-pi, pi]

    Parameters:
    angle (float)   -- unwrapped angle

    Returns:
    angle (float)   -- wrapped angle
    """
    while angle >= math.pi:
        angle -= 2*math.pi

    while angle <= -math.pi:
        angle += 2*math.pi
    return angle


In [145]:
def convert_gps_to_xy(lat_gps, lon_gps, lat_origin, lon_origin):
    """Convert gps coordinates to cartesian with equirectangular projection

    Parameters:
    lat_gps     (float)    -- latitude coordinate
    lon_gps     (float)    -- longitude coordinate
    lat_origin  (float)    -- latitude coordinate of your chosen origin
    lon_origin  (float)    -- longitude coordinate of your chosen origin

    Returns:
    x_gps (float)          -- the converted x coordinate
    y_gps (float)          -- the converted y coordinate
    """
    x_gps = EARTH_RADIUS*(math.pi/180.)*(lon_gps - lon_origin)*math.cos((math.pi/180.)*lat_origin)
    y_gps = EARTH_RADIUS*(math.pi/180.)*(lat_gps - lat_origin)

    return x_gps, y_gps


In [146]:
def propogate_state(x_t_prev, u_t):
    """Propogate/predict the state based on chosen motion model

    Parameters:
    x_t_prev (np.array)  -- the previous state estimate
    u_t (np.array)       -- the current control input

    Returns:
    x_bar_t (np.array)   -- the predicted state
    """
    """STUDENT CODE START"""
    #Xt = [xg, yg, yaw, yaw_t-1, Vx, Vy, omega]'
    angle_correction = x_t_prev[2]
    #angle_correction = wrap_to_pi(x_t_prev[2])

    A_matrix = np.array([[1, 0, 0, 0, DT, 0, 0],
                         [0, 1, 0, 0, 0, DT, 0],
                         [0, 0, 1, 0, 0, 0, DT],
                         [0, 0, 1, 0, 0, 0, 0 ],
                         [0, 0, 0, 0, 1, 0, 0 ],
                         [0, 0, 0, 0, 0, 1, 0 ],
                         [0, 0, 1/DT, -1/DT, 0, 0, 0]])
    
    #angle_correction = wrap_to_pi(2*np.pi - x_t_prev[2])
    
    
    #angle_correction = wrap_to_pi(-x_t_prev[2])


    B_matrix = np.array([[1/2*np.square(DT)*np.cos(angle_correction), 0],
                     [-1/2*np.square(DT)*np.sin(angle_correction), 0],
                     [0, 0],
                     [0, 0],
                     [DT*np.cos(angle_correction), 0], 
                     [-DT*np.sin(angle_correction), 0],
                     [0, 0]])

    x_bar_t = A_matrix@x_t_prev + B_matrix@u_t
    x_bar_t[2] = wrap_to_pi((A_matrix@x_t_prev)[2] + (B_matrix@u_t)[2])
    x_bar_t[3] = wrap_to_pi((A_matrix@x_t_prev)[3] + (B_matrix@u_t)[3])
    
    
    x_bar_t[6] = wrap_to_pi(x_t_prev[2] - x_t_prev[3]) / DT
    #x_bar_t[6] = (wrap_to_pi(x_t_prev[2]) - wrap_to_pi(x_t_prev[3])) / DT
    #x_bar_t[2] = wrap_to_pi(x_bar_t[2])

    """STUDENT CODE END"""

    return x_bar_t

## State Representation

### At each time step, the ith row of the particle matrix has the states corresponding to the ith particle 

In [153]:
def getWeight(x_t, z_t, covar_z_t):
    
    estimated = x_t[0:3]
    
    weight = multivariate_normal.pdf(z_t, mean=estimated, cov=covar_z_t)
    
#     if weight == 0:
#         print("getWeight")
#         print('x_t')
#         print(x_t)
#         print('estimated')
#         print(estimated)
#         print('z_t')
#         print(z_t)
#         print('weight')
#         print(weight)
#         print("wtf")
    
    return weight
    

In [148]:
def prediction_step(x_t_prev, u_t, covar_u_t, z_t, covar_z_t):
    '''
    z_t is the bot's position in the global frame
    
    '''
    
    x_t_bar = np.zeros(x_t_prev.shape)
    weights = np.zeros(len(x_t_prev))
    
    for i in range(len(x_t_prev)):
        perturbed_u_t = np.zeros(2)
        
        # Get previous state
        prev_state = x_t_prev[i]
        
        # Perturb control input
        perturbed_u_t[0] = u_t[0] + np.random.normal(loc = 0, scale = np.sqrt(covar_u_t[0][0]))
        perturbed_u_t[1] = u_t[1] + np.random.normal(loc = 0, scale = np.sqrt(covar_u_t[1][1]))
        
        # Get new state using perturbed control input
        new_state = propogate_state(prev_state, perturbed_u_t)
        new_weight = getWeight(new_state, z_t, covar_z_t)
        
        # Store
        x_t_bar[i] = new_state
        weights[i] = new_weight
        
#     if np.isnan(np.sum(weights)):
#         print("hello")
#         print(x_t_prev)
#         print(z_t)
#         print(weights)
#         print('hiiii')
    
    weights_sum = weights/np.sum(weights)
    
#     if np.isnan(np.sum(weights_sum)):
#         print(x_t_prev)
#         print(z_t)
#         print(weights)
#         print(np.sum(weights))
    #print(np.sum(weights))
        
    return x_t_bar, weights_sum
        
    

In [149]:
def correction_step(x_bar_t, weights):
    
    x_t = np.zeros(x_bar_t.shape)
    #weights = np.zeros(len(x_bar_t))
        
    x_ind = np.arange(len(x_bar_t))
    
    for i in range(len(x_bar_t)):
        
        #print(x_ind.shape)
        #print(weights.shape)
        #print("correction")
        #print(np.sum(weights))
        
#         if np.isnan(np.sum(weights)):
#             print(weights)
        x_t_index = np.random.choice(a = x_ind, p = weights)
        
        x_t[i] = x_bar_t[x_t_index]
        #weights[i] = weights[x_t_index]
        
    return x_t
        

In [150]:
def initializeParticles(numParticles, numStates):
    
    states = np.zeros([numParticles, numStates])
    
    x_col = np.random.normal(loc = 0, scale = 0.1, size = numParticles)
    y_col = np.random.normal(loc = 0, scale = 0.1, size = numParticles)
    
    yaw_col = np.random.normal(loc = 0, scale = 0.01, size = numParticles)
    #yaw_col = np.where(yaw_col < 0, yaw_col + np.pi*2, yaw_col)
    
    states[:, 0] = x_col
    states[:, 1] = y_col
    states[:, 2] = yaw_col
    states[:, 3] = yaw_col
        
    return states
    
    

In [151]:
def main():
    
    filename = '2020_2_26__17_21_59_filtered'
    data, is_filtered = load_data(filename)
    
    # Load data into variables
    x_lidar = data["X"]
    y_lidar = data["Y"]
    z_lidar = data["Z"]
    time_stamps = data["Time Stamp"]
    lat_gps = data["Latitude"]
    lon_gps = data["Longitude"]
    yaw_lidar = data["Yaw"]
    pitch_lidar = data["Pitch"]
    roll_lidar = data["Roll"]
    x_ddot = data["AccelX"]
    y_ddot = data["AccelY"]
    
    lat_origin = lat_gps[0]
    lon_origin = lon_gps[0]
    
    # Initialize filter
    numParticles = 1000
    numStates = 7
    x_t_prev = initializeParticles(numParticles, numStates)
    
    # Store particles and gps
    particles = np.empty((numParticles, numStates, len(time_stamps)))
    gps_estimates = np.empty((2, len(time_stamps)))
    
    index = 0
    
    for t, _ in enumerate(time_stamps):
        #print('Index')
        #print(index)
        index = index + 1
        
        u_t = np.array([x_ddot[t], y_ddot[t]])
        
        converted_yaw = wrap_to_pi(yaw_lidar[t] * np.pi / 180)
        
        z_t = np.array([5 - (x_lidar[t]*(-np.sin(converted_yaw)) + y_lidar[t]*(np.cos(converted_yaw))),
                        -5 - (x_lidar[t]*(-np.cos(converted_yaw)) - y_lidar[t]*(np.sin(converted_yaw))),
                        converted_yaw])
        
        # Prediction Step
        x_bar_t, weights = prediction_step(x_t_prev, u_t, covar_u_t, z_t, covar_z)
        #print(np.sum(weights))
        
        # Correction Step
        x_t = correction_step(x_bar_t, weights)
        
        
        # Explicitly set x_prev_t to x_t
        x_prev_t = x_t
        
        # Store shit
        particles[:, :, t] = x_t

        x_gps, y_gps = convert_gps_to_xy(lat_gps=lat_gps[t],
                                         lon_gps=lon_gps[t],
                                         lat_origin=lat_origin,
                                         lon_origin=lon_origin)
        gps_estimates[:, t] = np.array([x_gps, y_gps])
        
    return particles, gps_estimates
    

In [154]:
particles, gps_estimates = main()

/Users/christinalau/anaconda/envs/E190/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


ValueError: probabilities contain NaN

In [ ]:
test = np.ones(1000)/1000

In [101]:
np.random.choice(a = np.arange(1000), p = test)

789